In [ ]:
(ql:quickload "kenzo")

In [ ]:
(use-package "CAT")

In [ ]:
(cat-init)

# Simplicial Sets

## Introduction

A *simplicial set* (**Hilton &  Wylie**, *Homology theory*, Cambridge University Press, 1967) $K$ is a union $K={\displaystyle{\bigcup_{q\ge 0}{K^q}}}$, where the $K^q$ are disjoints sets, together with functions\index{simplicial sets}

$$\partial_i^q: K^q \longrightarrow K^{q-1},\quad q>0,\quad i=0,\ldots , q,$$

$$\eta_i^q: K^q \longrightarrow K^{q+1},\quad q\ge 0,\quad i=0,\ldots ,q,$$

subject to the relations

$$
\begin{align}
\partial_i^{q-1}\partial_j^q   & = & \partial_{j-1}^{q-1}\partial_i^q, &   i < j \cr
\eta_i^{q+1}\eta_j^q       & = & \eta_j^{q+1}\eta_{i-1}^q,    & i > j    \cr
\partial_i^{q+1}\eta_j^q     & = & \eta_{j-1}^{q-1}\partial_i^q,   & i < j  \cr
\partial_i^{q+1}\eta_i^q     & = & \partial_{i+1}^{q+1}\eta_i^q    & Identity,   \cr
\partial_i^{q+1}\eta_j^q     & = & \eta_j^{q-1}\partial_{i-1}^q,   &  i > j+1.\cr 
\end{align}
$$

An element of $K^q$ is an (*abstract*) $q$-simplex of $K$ and the functions $\partial$ and $\eta$ are respectively the *face operators* and the *degeneracy operators*. Formally, their action on a simplex is very simple. For instance, if a $q$--simplex is an ordered set of vertices, like $\lbrace v_0, v_1, \ldots,v_i,\ldots, v_q \rbrace$, the rules are the following (we omit the indication of the dimension of the simplex):

$$
\begin{eqnarray*}
\partial_i \lbrace v_0, v_1, \ldots,v_i,\ldots, v_q \rbrace & = &
           \lbrace v_0, v_1, \ldots,{\bf v_{i-1}}, {\bf v_{i+1}},\ldots, v_q \rbrace, \\
\eta_i \lbrace v_0, v_1, \ldots,v_i,\ldots, v_q \rbrace & = &
      \lbrace v_0, v_1, \ldots,{\bf v_i},{\bf v_i},\ldots, v_q \rbrace.
\end{eqnarray*}
$$

The operators $\partial_i^q$ will be used hereafter to define the boundary operator $d^q$ for the $q$-component of the associated chain complex:

$$d^q = \sum_{i=0}^q { (-1)^i \partial_i^q}.$$

The image of a simplex under some $\eta$ is called *degenerate*, because it
is not directly implied  in the realization of the simplicial set.

### Example

Let us take the small example *diabolo* from the chapter 1. For a better understanding, it is convenient to use a list representation for the simplices. So the set $K^0$ of 0-simplices (vertices) is:

$$\lbrace (0), (1),\ldots (5) \rbrace.$$

The set $K^1$ of 1-simplices contains the set of regular simplices:

$$\lbrace (0\  1), (0\  2), (1\  2), (2\  3), (3\  4), (3\  5), (4\  5) \rbrace,$$

but also the degenerate 1-simplices:

$$\lbrace (0\ 0), (1\ 1), (2\ 2),\ldots,(5 \ 5) \rbrace.$$

The set $K^2$ of 2-simplices contains the regular singleton:  

$$\lbrace ( 3\  4\  5) \rbrace$$

but also the degenerate 2-simplices:

$$\lbrace (0\ 0\ 0), \ldots, (5\ 5\ 5), (0\ 0\ 1), (0\ 0\  2), (1\ 1\ 2),\ldots, (4\ 5\ 5)\rbrace.$$

The sets $K^q$ ($q >2$) contain only degenerate simplices. With this  naming of the elements of a $q$-simplex, the action of $\partial$ and $\eta$ is now clear. For instance

$$\partial_0 (0\  1) = (1),\  \partial_1 (0\  1) = (0),$$

$$\eta_0 (0\  1) = (0\  0\  1),\ 
  \eta_1 (1\  2)= (1\  2\  2),\ \eta_2(3\ 4\ 5)=(3\ 4\ 5\ 5). $$

By the way, we see that a simplex is non-degenerate if the list is strictly increasing.

## Notion of abstract simplex in this Lisp implementation

In this implementation, on account of the essential following property: *any simplex
may be expressed in an unique way as a possibly iterated degeneracy of a non--degenerate simplex*, we call an **abstract simplex**, (in short **absm**), a pair consisting of:
- a (possibly iterated) degeneracy operator,
- a "geometric" simplex, i.e. a non--degenerate simplex.
For instance, if $\sigma$ is a non-degenerate simplex, and $\sigma'$ is the degenerate simplex  $\eta_3\eta_1\sigma$, the corresponding abstract simplices are respectively $\lbrace \emptyset, \sigma \rbrace$ and $\lbrace \eta_3\eta_1, \sigma \rbrace$.

An abstract simplex is represented internally in the system by the following lisp object:
$$\texttt{(:absm}\quad\mathit{dgop}\quad\texttt{.}\quad\mathit{gmsm}\texttt{)}$$
where,
1. *dgop* is a non-negative integer coding a *strictly decreasing integer list*. This strictly  decreasing list of integers represents a sequence of $\eta$ operators and is coded as a **unique** integer with the following convention: a number $i$ in the sequence is the $i+1$--th binary bit of a machine word. The null list is coded as the number 0 whereas the list ``(0)``, i.e. the $\eta_0$ operator, is coded as the integer ``1``, etc. Two functions are provided to help the user for the transformation in both directions.
2. *gmsm* is a geometric simplex,  i.e. any kind lisp object modelizing a non-degenerate simplex. For practical reason, a  type ``GMSM``, (any kind of lisp object), has been defined.

The simplest constructor is the macro ``(absm (dgop-ext-int`` *dgop-list*``)``  *gmsm*``)``, where *dgop-list* is a strictly decreasing integer list. A type ``ABSM`` has been defined. The associated printing method prints such an object under the form:
$$\texttt{<AbSm}\quad\mathit{ext-dgop}\quad\mathit{gmsm}{\texttt >}$$
where *ext-dgop* shows clearly the sequence of the operators $\eta_i$ (see the examples). The accessor functions for the components are the macros ``dgop`` and ``gmsm``.

### Simple functions for abstract simplices

``dgop-ext-int`` *ext-dgop* *[Function]*
>Code on an integer the valid list representing the degeneracy operator *ext-dgop*.

``dgop-int-ext`` *dgop* *[Function]*
> Give the list representing a degeneracy operator from the integer *dgop*.

``absm`` *dgop gmsm* *[Macro]*
> Create an abstract simplex, using directly the coded representation, *dgop*, of the degeneracy operator.

``dgop`` *absm* *[Macro]*
> Get the integer code of the internal representation of the degeneracy operator of the abstract simplex *absm*.
 
``gmsm`` *absm* *[Macro]*
> Get the basic non-degenerate simplex part of the abstract simplex ``absm``.

``absm-p`` *object* *[Predicate]*
>Test if *object* is an abstract simplex.

### Examples

Let us suppose that ``:sigma`` "is" a geometric simplex, we may construct the abstract simplices of the beginning of the section.

In [ ]:
(dgop-ext-int '(0))

In [ ]:
(dgop-ext-int '(4 0))

In [ ]:
(dgop-int-ext 63)

In [ ]:
(dgop-ext-int '(2 2))

In [ ]:
(def asigma (absm 0 :sigma))

In [ ]:
(def asigma-prime (absm (dgop-ext-int '(3 1)) :sigma))

In [ ]:
(absm-p asigma-prime)

The following  command gives the sixth degeneracy of the vertex 0 represented as ``(0)``.

In [ ]:
(def deg-6 (absm (dgop-ext-int '(5 4 3 2 1 0)) '(0)))

In [ ]:
(dgop deg-6)

In [ ]:
(gmsm deg-6)

## Representation of a simplicial set

A simplicial set is implemented as an instance of the class ``SIMPLICIAL-SET``, subclass of the class ``COALGEBRA``.

```lisp
(DEFCLASS SIMPLICIAL-SET (coalgebra)
    ((face :type face :initarg :face :reader face1)))

```

This class inherits also from the class ``CHAIN-COMPLEX`` and has one slot of its own:

>``face``, a lisp function  computing any face of any geometric simplex of the simplicial set.
This is a function with 3 arguments: a face index (a non--negative integer, ``indx``), a simplex dimension (a positive integer, ``dmns``) and a geometric simplex (``gmsm``), the $indx$-th face of which is looked for. Whatever the face simplex is, degenerate or not, the simplex returned by this function must be an **abstract** simplex, i.e. an ``ABSM`` object.

A printing method has been associated to the class ``SIMPLICIAL-SET`` and the external representation of  an instance is a string like ``[K``*n* ``Simplicial-Set]``, where $n$ is the number plate of the Kenzo object.

## The function ``build-smst``

To facilitate the construction of instances of the class ``SIMPLICIAL-SET``, the software ``Kenzo`` provides the function ``build-smst`` defined with keyword parameters:

``build-smst``
>``:cmpr`` *cmpr* ``:basis`` *basis* ``:bspn`` *bspn* ``:face`` *face*  
``:face*`` *face$*$* ``:intr-bndr`` *intr-bndr* ``:bndr-strt`` *bndr-strt*  
``:intr-dgnl`` *intr-dgnl* ``:dgnl-strt`` *dgnl-strt* ``:orgn`` *orgn*

The returned value is an object of type ``SIMPLICIAL-SET``. This function assigns an identification number ``idnm`` to the Kenzo object instance in a sequential way and pushes this object on the list of already  created simplicial sets instances, ``*smst-list*``.

The keywords parameters are:
- *cmpr*, a comparison function for generators. 
- *basis*, if the simplicial set is effective, this argument must be the lisp function returning the  list of non-degenerate simplices  of the simplicial set, in a given dimension; if the simplicial set is locally effective, this argument must be the keyword ``:locally-effective``.
- *bspn*, the lisp representation of the base point (an item of type ``GMSM``). This is in fact the slot ``bsgn`` of the associated chain complex.
- *face*, a lisp function for the face operators.
- $face*$, a lisp function for the face operators, returning the symbol ``:degenerate`` if the corresponding face is degenerate.
- *intr-bndr*, a lisp function for the differential of the associated chain complex.
- *bndr-strt*, the strategy (``:cmbn`` or ``:gnrt``) for the function *intr-bndr*.
- *intr-dgnl*, a lisp function for the coproduct of the coalgebra.
- *dngl-strt*, the strategy (``:cmbn`` or ``:gnrt``) for the function *intr-dgnl*.
- *org*, the comment list, adequately chosen.

The three arguments $face*$,  *intr-bndr* and *intr-dgnl* are not mandatory. Only the function *face* is necessary to define precisely the simplicial set. From the function *face*, we know that we may construct:
- The differential for the associated chain complex, according to the formula:
$$d^q = \sum_{i=0}^q { (-1)^i \partial_i^q}.$$
- The coproduct of the underlying coalgebra (the Alexander-Whitney diagonal, $\Delta$) according to the formula:
$$\Delta(\sigma)= \sum_{i=0}^{n}{{\tilde{\delta}}^{(i)}\sigma \otimes \delta_0^{(i)} \sigma},$$
where the power $(i)$ denotes the $i$-th composition of the operator and $\tilde{\delta}$ is the
*last face* operator, so that
$${\tilde{\delta}}^{(i)}=\delta_{n-i+1}\circ \cdots \circ \delta_{n-1}\circ \delta_n.$$

But, as the differential ignore degenerate faces, it may be interesting for the user to provide a face function, weaker than the function *face* but sufficient and more efficient to compute the boundary of a simplex. It is the raison d'etre of the parameter $face*$ which may be used as an alternative to *face*. In both cases, the strategy is set to ``:gnrt`` by the system. It may also arrive that the user knows a particularly simple form for the differential, he may then use the parameter *intr-bndr*. In the computation of differentials in the associated chain complex, this function will be used instead of the canonical differential computed from the face function. In this last case, the user is required to provide the strategy. Same remark for the parameter *intr-dgnl*.

### Examples

As first example, we give the construction of the simplicial set $\Delta^n$  corresponding to the 
standard $n$-simplex. This is given only as example. The program ``Kenzo`` provides a function
for the same purpose which will be described later. For every simplex, we shall use a list representation as in our first example ``diabolo``. The various keyword arguments for the call to ``build-smst``, are:
1. *bspn*, the list ``(0)``, i.e. the representation of the vertex 0.
2. *cmpr*, the lisp function  ``#'l-cmpr``, since this function is adequate to compare the lists representing the simplices.
3. *basis*, the lisp function returning the list of all possible non-degenerate simplices in some dimension. Here, this is the combinatory function returning the set of all the lists of $p+1$ objects taken among $n+1$. Such a function is for instance the function  (``delta-inj`` *p n*) whose definition is given hereafter. So, the ``:basis`` keyword argument is simply:
```lisp
#'(lambda(dmn) (delta-inj dmn n))
```.
Note that here, ``n`` is a global parameter.
4. *face*, the lisp function for the face operators (``face-delta-n``). It consists simply in removing the $i$-th element of the list describing a simplex ($i$ starting at 0).
5. *org* will be a comment.

In [ ]:
(defun delta-inj (m n)
  (declare (type fixnum m n))
  (cond ((> m n) +empty-list+)
        ((zerop m) (mapcar #'list (<a-b> 0 n)))
        (t (mapcan #'(lambda (list)
                       (declare (type list list))
                       (mapcar #'(lambda (k)
                                   (declare (type fixnum k))
                                   (cons k list))
                                 (<a-b< 0 (first list))))
                     (delta-inj (1- m) n)))))

In [ ]:
(delta-inj 0 3)

In [ ]:
(delta-inj 1 3)

In [ ]:
(delta-inj 3 3)

(def face-delta-n
  #'(lambda(i dmn gsm)
      (declare(ignore dmn))
      (absm 0 (append (subseq gsm 0 i)
                      (subseq gsm (1+ i))) )))

To create $\Delta^n$ for any order $n$, we embed the call of ``build-smst`` in a function having $n$ as parameter.

In [ ]:
(defun delta-n(n)
  (declare (type fixnum n))
        (build-smst :cmpr #'l-cmpr
                    :basis #'(lambda(dmn)(delta-inj dmn n))
                    :bspn '(0)
                    :face face-delta-n
                    :orgn `(delta-n ,n)))

In [ ]:
(def delta4 (delta-n 4))

In [ ]:
(bspn delta4)

In [ ]:
(basis delta4 0)

In [ ]:
(basis delta4 3)

A second example is the contruction of the simplicial complex  $\Delta^\mathbb{N}$ freely generated by the positive integers. The construction of the ``SIMPLICIAL-SET`` instance to implement this simplicial set is quite similar to that of $\Delta^n$, but in this case the ``:basis`` keyword argument is  the keyword ``:locally-effective``, since the sets $K^q$ are infinite. Whence, the definition of $\Delta^\mathbb{N}$:

In [ ]:
(def delta-infty
        (build-smst :cmpr #'l-cmpr
                    :basis :locally-effective
                    :bspn '(0)
                    :face face-delta-n
                    :orgn `(delta-infinity)))

A third example is the construction of the simplicial set corresponding to ``diabolo``. This simplicial set is a  sub-complex of $\Delta^2$. The ``:face`` function is exactly the same as in the examples above and the ``:basis`` function returns the sub-simplices of ``diabolo``, by enumeration. 

In [ ]:
(def diabolo-ss 
        (build-smst :cmpr #'l-cmpr
                    :basis  #'(lambda (dmn)
                                (case dmn
                                      (0 '((0)(1)(2)(3)(4)(5)))
                                      (1 '((0 1)(0 2)(1 2)(2 3)(3 4)(3 5)(4 5)))
                                      (2 '((3 4 5)))))
                    :bspn '(0)
                    :face face-delta-n
                    :orgn '(simplicial set for diabolo)))

## A first set of helpful functions on simplicial sets